# A script to pull images from the Google Image Search

Please read the more detailed description of this script at https://github.com/dusanstanojeviccs/python-google-image-downloader

Short description of the script:

Before running make sure you have Chrome installed and make sure you put the correct chromedriver or chromedriver.exe file (if on Windows) in the same folder as this file.

Runnning this file will download images of cats by running the search for the keyword "cat" into the "cat" folder, to change those change the variables ```query``` and ```class_name``` respectively. You can also change how many images you would like to download change the variable ```image_count```. To change the maximum number of tries to load more images before if the image_count has not been reached you can change the variable ```max_loops_with_no_images```.


In [1]:
!pip install selenium Pillow

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [2]:
!chmod 755 chromedriver

In [3]:
from selenium import webdriver
import os
from time import sleep
import base64
from PIL import Image
from io import BytesIO
import requests

from selenium.webdriver.chrome.options import Options  


In [4]:
driver_path = "./chromedriver"

if os.name == 'nt':
    driver_path = "./chromedriver.exe"

query = "cat"
class_name = "cat"
image_count = 10
max_loops_with_no_images = 10

chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1200")
chrome_options.add_argument("--ignore-certificate-errors")


browser = webdriver.Chrome(driver_path, chrome_options=chrome_options)
browser.get("https://www.google.com/search?q=" + query + "&source=lnms&tbm=isch&sa=X&biw=1029&bih=818&dpr=2")

sleep(2)

downloaded_elements = []
processeed_elements = []
downloaded = 0

try:
    os.mkdir(class_name)
except OSError:
    print("The directory existed")

loops_with_no_images = 0
while len(downloaded_elements) < image_count:
    downloaded_in_current_loop = 0

    browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    show_more_button = browser.find_elements_by_css_selector("[value='Show more results']")

    for button in show_more_button:
        if button.is_displayed() and button.is_enabled():
            button.click()

    sleep(2)

    elements = browser.find_elements_by_css_selector("a.rg_l img")
    for i in range(len(elements)):
        element = elements[i]
        if element in processeed_elements:
            continue

        processeed_elements.append(element)

        src = element.get_attribute("src")
        if not src:
            src = element.get_attribute("data-src")

        if not src:
            print(element.get_attribute('outerHTML'))
            print(element)
            print(src)
        
        if src and src.startswith("data:"):
            data = src.split(",")[1]
            im = Image.open(BytesIO(base64.b64decode(data)))
            im.save(class_name + "/" + str(len(downloaded_elements)) + '.png', 'PNG')
            downloaded_in_current_loop = downloaded_in_current_loop + 1
            downloaded_elements.append(element)
        elif src:
            response = requests.get(src)
            im = Image.open(BytesIO(response.content))
            im.save(class_name + "/" + str(len(downloaded_elements)) + '.png', 'PNG')
            downloaded_in_current_loop = downloaded_in_current_loop + 1
            downloaded_elements.append(element)

        if len(downloaded_elements) > image_count:
            break
    
    if downloaded_in_current_loop == 0:
        loops_with_no_images = loops_with_no_images + 1
    else:
        loops_with_no_images = 0

    if loops_with_no_images > max_loops_with_no_images:
        print("FOUND NO IMAGES IN " + str(max_loops_with_no_images) + " LOOPS :O")
        break

print("I AM PRETTY MUCH DONE ", len(downloaded_elements))

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


('I AM PRETTY MUCH DONE ', 11)
